In [ ]:
import numpy as np
import pandas as pd
import os


MODEL KLASYCZNY ML


In [ ]:
from scipy.sparse import load_npz
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import joblib

X_train = load_npz('data/processed/X_train_tfidf.npz')
X_test  = load_npz('data/processed/X_test_tfidf.npz')

y_train = np.load('data/processed/y_train.npy')
y_test  = np.load('data/processed/y_test.npy')

X_train.shape, X_test.shape


((41611, 20000), (37087, 20000))

In [ ]:
ml_model = LogisticRegression(
    max_iter=1000,
    n_jobs=-1
)

ml_model.fit(X_train, y_train)


LogisticRegression(max_iter=1000, n_jobs=-1)

In [ ]:
y_pred_ml = ml_model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_ml))
print(classification_report(y_test, y_pred_ml))


Accuracy: 0.8766953379890527
              precision    recall  f1-score   support

           0       0.88      0.87      0.87     18216
           1       0.87      0.89      0.88     18871

    accuracy                           0.88     37087
   macro avg       0.88      0.88      0.88     37087
weighted avg       0.88      0.88      0.88     37087



In [ ]:
os.makedirs('models', exist_ok=True)
joblib.dump(ml_model, 'models/logistic_regression.pkl')


['models/logistic_regression.pkl']

SIEĆ NEURONOWA OD ZERA

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset


In [ ]:
X_train_dense = torch.tensor(X_train.toarray(), dtype=torch.float32)
X_test_dense  = torch.tensor(X_test.toarray(), dtype=torch.float32)

y_train_t = torch.tensor(y_train, dtype=torch.float32)
y_test_t  = torch.tensor(y_test, dtype=torch.float32)

train_ds = TensorDataset(X_train_dense, y_train_t)
test_ds  = TensorDataset(X_test_dense, y_test_t)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)


In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_nn = SimpleNN(X_train.shape[1]).to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model_nn.parameters(), lr=0.001)

epochs = 5

for epoch in range(epochs):
    model_nn.train()
    total_loss = 0

    for Xb, yb in train_loader:
        Xb, yb = Xb.to(device), yb.to(device).unsqueeze(1)

        optimizer.zero_grad()
        preds = model_nn(Xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss:.4f}")


Epoch 1/5 - Loss: 214.6881
Epoch 2/5 - Loss: 113.7261
Epoch 3/5 - Loss: 66.6471
Epoch 4/5 - Loss: 29.9788
Epoch 5/5 - Loss: 10.0377


In [ ]:
model_nn.eval()
with torch.no_grad():
    preds = model_nn(X_test_dense.to(device)).cpu().numpy()
    preds = (preds > 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test, preds))
print(classification_report(y_test, preds))


Accuracy: 0.8520775473885728
              precision    recall  f1-score   support

           0       0.87      0.83      0.85     18216
           1       0.84      0.88      0.86     18871

    accuracy                           0.85     37087
   macro avg       0.85      0.85      0.85     37087
weighted avg       0.85      0.85      0.85     37087



In [ ]:
torch.save(model_nn.state_dict(), 'models/nn_from_scratch.pt')


TRANSFORMER (FINE-TUNING)

In [ ]:
!pip install -q transformers datasets accelerate


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset


In [ ]:
train_df = pd.read_csv('data/processed/train_transformer.csv')
test_df  = pd.read_csv('data/processed/test_transformer.csv')

train_df = train_df.sample(3000, random_state=42)
test_df  = test_df.sample(500, random_state=42)



train_df.head()


,clean_text,label
7980,cline is at her best in french whenever people...,1
22413,depressing instead of funny a huge fan has bee...,0
19354,megaland popup playset is fantastic it is amaz...,1
3640,crap this is a crap movie totally unbelievable...,0
41094,its one of those movies this is one of those m...,1


In [ ]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(
        batch['clean_text'],
        truncation=True,
        padding='max_length',
        max_length=128
    )

train_ds = Dataset.from_pandas(train_df)
test_ds  = Dataset.from_pandas(test_df)

train_ds = train_ds.map(tokenize, batched=True)
test_ds  = test_ds.map(tokenize, batched=True)

train_ds = train_ds.rename_column("label", "labels")
test_ds  = test_ds.rename_column("label", "labels")

train_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
test_ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
transformer_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="models/transformer",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="results/logs",
    report_to="none"
)


trainer = Trainer(
    model=transformer_model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=tokenizer
)

trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,0.284332


TrainOutput(global_step=188, training_loss=0.3548125814884267, metrics={'train_runtime': 2308.4153, 'train_samples_per_second': 1.3, 'train_steps_per_second': 0.081, 'total_flos': 99350548992000.0, 'train_loss': 0.3548125814884267, 'epoch': 1.0})

In [ ]:
trainer.evaluate()


{'eval_loss': 0.28433215618133545,
 'eval_runtime': 113.7745,
 'eval_samples_per_second': 4.395,
 'eval_steps_per_second': 0.281,
 'epoch': 1.0}

In [ ]:
trainer.save_model("models/transformer")
tokenizer.save_pretrained("models/transformer")


('models/transformer/tokenizer_config.json',
 'models/transformer/special_tokens_map.json',
 'models/transformer/vocab.txt',
 'models/transformer/added_tokens.json',
 'models/transformer/tokenizer.json')